# Importing the required libraries

In [ ]:
import numpy as np
import wave
import matplotlib.pyplot as plt
%matplotlib inline
from glob import glob
import random
import math
import struct
from keras.models import *
from keras.layers import *
from keras.callbacks import *
import os
from tabulate import tabulate
from tensorflow import keras
from keras import optimizers, losses, activations, models
import keras
from IPython.core.display import display, HTML
import tensorflow as tf
from sklearn.metrics import confusion_matrix

In [ ]:
display(HTML("<style>.container { width:80% !important; }</style>"))

# Check for available GPU

In [ ]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

# Visualizing a random test file

In [ ]:
test_file = "{}/Train/0/syntheticDopplerAudioCombined_0_892425.wav".format(DATA_DIR)

with wave.open(test_file, 'rb') as f:
    params = f.getparams()
    nchannels, sampwidth, framerate, nframes = params[:4] #Parameters of selected audio file
    print(nchannels, sampwidth, framerate, nframes)
    strData = f.readframes(nframes) 

waveData = np.frombuffer(strData, dtype=np.int16)        #Binary to wave conversion
waveData_norm = waveData*1.0/(np.max(np.abs(waveData))) #Wave data

n = 0.03*np.random.normal(0, 1, waveData_norm.shape)  #Noise to the selected data
waveData_norm = n + waveData_norm         #Adding noise to waveform

time = np.arange(0,nframes)*(1.0/framerate)
plt.plot(time, waveData_norm)
plt.xlabel("Time (s)")
plt.ylabel("Amplitude")
plt.title("Single channel wavedata")
plt.grid('on')
plt.show()

# Organizing the data into train, valiadation, and test sets

In [ ]:
MAX_FRAME = 40000 #Length of input frame for 8kHz data
LABELS = ['0', '1', '2', '3', '4'] #Target classes for classification task

N_CLASS = len(LABELS) #Number of classes for classification task

train_files, test_files, val_files = [], [], []

#####
# "Path_to_data" in all data paths must be correctly set 
#####

DATA_DIR_Train = r'C:\Users\Path_to_data\DopplerDataset\Shuffled_data\Train'
DATA_DIR_Test = r'C:\Users\Path_to_data\DopplerDataset\Shuffled_data\Test'
DATA_DIR_Validation = r'C:\Users\Path_to_data\DopplerDataset\Shuffled_data\Validation'

## Training Set
for f in glob('{}/*/*.wav'.format(DATA_DIR_Train)):
    train_files.append(f)   

## Validation Set
for f in glob('{}/*/*.wav'.format(DATA_DIR_Validation)):
    val_files.append(f)
    
## Test Set
for f in glob('{}/*/*.wav'.format(DATA_DIR_Test)):
    test_files.append(f) 

## Suffling the Training and Validation sets to remove bias in dataset
random.shuffle(train_files)
random.shuffle(val_files)

# Reading and normalizing wave data

In [ ]:
def add_noise(w):
    noise = 0.03*np.random.normal(0, 1, w.shape)
    w = noise + w
    return w

## Functions to read all the data in Train, Validation, and Test sets and do normalization

def get_wave_norm(file):
    with wave.open(file, 'rb') as f:
        params = f.getparams()
        nchannels, sampwidth, framerate, nframes = params[:4]
        data = f.readframes(nframes)
    data = np.frombuffer(data, dtype=np.int16)
    data = data/(np.max(np.abs(data)))
    return data

def get_wave(file):
    data = get_wave_norm(file)
    wave_data = np.zeros((MAX_FRAME, ))
    wave_len = min(MAX_FRAME, data.shape[0])
    wave_data[:wave_len] = data[:wave_len]
    return np.expand_dims(wave_data, axis=1)

# Constructing the input and target: Train and Validation sets

In [ ]:
# Populating the input and target data - Training
x_train = np.zeros((L, MAX_FRAME, 1), dtype=np.float32)
y_train = np.zeros((L, N_CLASS), dtype=np.uint8)
for i in range(L):
    f_train = train_files[i]
    x_train[i] = get_wave(f_train)
    label = f_train[len(DATA_DIR_Train) + 1:]
    label = label[:label.index('\\')]
    label_idx = LABELS.index(label[0])
    y_train[i, label_idx] = 1
     
# Populating the input and target data - Validation
L_val = len(val_files)
x_val = np.zeros((L_val, MAX_FRAME, 1), dtype=np.float32)
y_val = np.zeros((L_val, N_CLASS), dtype=np.uint8)
for i in range(L_val):
    f_val = val_files[i]
    x_val[i] = get_wave(f_val)
    label = f_val[len(DATA_DIR_Validation) + 1:]
    label = label[:label.index('\\')]
    label_idx = LABELS.index(label[0])
    y_val[i, label_idx] = 1

# Dataset balance check

In [ ]:
a1 = [1, 0, 0, 0, 0]
a2 = [0, 1, 0, 0, 0]
a3 = [0, 0, 1, 0, 0]
a4 = [0, 0, 0, 1, 0]
a5 = [0, 0, 0, 0, 1]

## Training set
cnt1_tr = 0
cnt2_tr = 0
cnt3_tr = 0
cnt4_tr = 0
cnt5_tr = 0        
for i in range(len(y_train)):
    if (np.array_equal(y_train[i], a1) == True):
        cnt1_tr += 1
    elif (np.array_equal(y_train[i], a2) == True):
        cnt2_tr += 1
    elif (np.array_equal(y_train[i], a3) == True):
        cnt3_tr += 1
    elif (np.array_equal(y_train[i], a4) == True):
        cnt4_tr += 1
    else:
        cnt5_tr += 1
table = [['Bubble Class', 'Frequency'], 
         ['Class 0', cnt1_tr], 
         ['Class 1', cnt2_tr], 
         ['Class 2', cnt3_tr], 
         ['Class 3', cnt4_tr], 
         ['Class 4', cnt5_tr]]
print(tabulate(table, tablefmt="grid"))

## Validation set
cnt1 = 0
cnt2 = 0
cnt3 = 0
cnt4 = 0
cnt5 = 0
for i in range(len(y_val)):
    if (np.array_equal(y_val[i], a1) == True):
        cnt1 += 1
    elif (np.array_equal(y_val[i], a2) == True):
        cnt2 += 1
    elif (np.array_equal(y_val[i], a3) == True):
        cnt3 += 1
    elif (np.array_equal(y_val[i], a4) == True):
        cnt4 += 1
    else:
        cnt5 += 1
table = [['Bubble Class', 'Frequency'], 
         ['Class 0', cnt1], 
         ['Class 1', cnt2], 
         ['Class 2', cnt3], 
         ['Class 3', cnt4], 
         ['Class 4', cnt5]]
print(tabulate(table, tablefmt="grid"))

# Network design

In [ ]:
callbacks = [keras.callbacks.ModelCheckpoint("best_model.h5", save_best_only=True, monitor="val_loss"),
             keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=5, min_lr=0.0001),
             keras.callbacks.EarlyStopping(monitor="val_loss", patience=15, verbose=1),]

inp = Input(shape=(MAX_FRAME, 1)) 
x = Convolution1D(32, kernel_size = 80, padding= 'same')(inp)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x1 = MaxPool1D(pool_size=3)(x)

x = Convolution1D(128, kernel_size = 3, padding= 'same')(x1)
x = BatchNormalization()(x)
x1 = Activation('relu')(x)
x1 = Dropout(rate=0.2)(x1)

x = Convolution1D(128, kernel_size = 3, padding= 'same')(x1)
x1 = BatchNormalization()(x)
x = GlobalAveragePooling1D()(x1)
x = Dense(128)(x)
x2 = Dense(128)(x)
x_mul = tf.keras.layers.Multiply()([x1, x2])
x_add_mul =  tf.keras.layers.Add()([x1, x_mul])

x3 = Activation('relu')(x_add_mul)
x4 = MaxPool1D(pool_size=3)(x3)

x4 = Convolution1D(128, kernel_size = 3, padding= 'same')(x4)
x4 = BatchNormalization()(x4)
x4 = Activation('relu')(x4)

x4  = Dropout(rate=0.2)(x4)
x4  = Convolution1D(128, kernel_size = 3, padding= 'same')(x4)
x11 = BatchNormalization()(x4)
x   = GlobalAveragePooling1D()(x11)
x   = Dense(128)(x)
x21 = Dense(128)(x)
x_mul1 = tf.keras.layers.Multiply()([x11, x21])
x_add_mul1 =  tf.keras.layers.Add()([x11, x_mul1])
x31 = Activation('relu')(x_add_mul1)
x41 = MaxPool1D(pool_size=3)(x31)

x41 = Convolution1D(256, kernel_size = 3, padding= 'same')(x41)
x41 = BatchNormalization()(x41)
x41 = Activation('relu')(x41)
x41 = Dropout(rate=0.2)(x41)
x41 = Convolution1D(256, kernel_size = 3, padding= 'same')(x41)
x111 = BatchNormalization()(x41)
x = GlobalAveragePooling1D()(x111)
x = Dense(256)(x)
x211 = Dense(256)(x)
x_mul11 = tf.keras.layers.Multiply()([x111, x211])
x_add_mul11 =  tf.keras.layers.Add()([x111, x_mul11])

x311 = Activation('relu')(x_add_mul11)
x411 = MaxPool1D(pool_size=3)(x311)

x411 = Convolution1D(256, kernel_size = 3, padding= 'same')(x411)
x411 = BatchNormalization()(x411)
x411 = Activation('relu')(x411)
x411 = Dropout(rate=0.2)(x411)
x411 = Convolution1D(256, kernel_size = 3, padding= 'same')(x411)
x1111 = BatchNormalization()(x411)
x = GlobalAveragePooling1D()(x1111)
x = Dense(256)(x)
x2111 = Dense(256)(x)
x_mul111 = tf.keras.layers.Multiply()([x1111, x2111])
x_add_mul111 =  tf.keras.layers.Add()([x1111, x_mul111])

x3111 = Activation('relu')(x_add_mul111)
x411 = MaxPool1D(pool_size=3)(x3111)


x = GlobalMaxPool1D(name = "Global_Pooling")(x411)
x = Dense(256)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(rate=0.2)(x)
dense_1 = Dense(N_CLASS, activation=activations.softmax)(x)
model = models.Model(inputs=inp, outputs=dense_1)
opt = tf.keras.optimizers.Adam(learning_rate=5e-5) 

model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
model.summary()

# Model training

In [ ]:
history = model.fit(x_train, y_train, 
                    validation_data = (x_val, y_val), 
                    epochs=30, batch_size=64, verbose=1,
                    callbacks=callbacks)

# Model accuracy and loss

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='lower right')
plt.show()
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper right')
plt.show()

# Loading the best model

In [ ]:
reconstructed_model = keras.models.load_model('best_model.h5')

# Fine-tuning the model with real data

In [ ]:
train_files_real, test_files_real, test_files_real_2, val_files_real= [], [], [], []

#Location of real data - Precordial region and subclavian veins
DATA_DIR_Train_Real = r'C:\Users\Path_to_data\Real_data\Percordial_real_data_shuffled_5s_70_10_20\Train'
DATA_DIR_Train_Real_2 = r'C:\Users\Path_to_data\Real_data\Subclavian_real_data_shuffled_5s_70_10_20\Train'

DATA_DIR_Test_Real = r'C:\Users\Path_to_data\Real_data\Percordial_real_data_shuffled_5s_70_10_20\Test'
DATA_DIR_Test_Real_2 = r'C:\Users\azarang\Desktop\Postdoc\deep_learning_audio_classification\Real_data\Subclavian_real_data_shuffled_5s_70_10_20\Test'

DATA_DIR_Validation_Real = r'C:\Users\Path_to_data\Real_data\Percordial_real_data_shuffled_5s_70_10_20\Validation'
DATA_DIR_Validation_Real_2 = r'C:\Users\Path_to_data\Real_data\Subclavian_real_data_shuffled_5s_70_10_20\Validation'


for f in glob('{}/*/*.wav'.format(DATA_DIR_Train_Real)):
    train_files_real.append(f)
print(len(train_files_real))
for f in glob('{}/*/*.wav'.format(DATA_DIR_Train_Real_2)):
    train_files_real.append(f) 
print(len(train_files_real))
    
for f in glob('{}/*/*.wav'.format(DATA_DIR_Test_Real)):
    test_files_real.append(f)
print(len(test_files_real))

for f in glob('{}/*/*.wav'.format(DATA_DIR_Test_Real_2)):
    test_files_real_2.append(f)
print(len(test_files_real_2))

for f in glob('{}/*/*.wav'.format(DATA_DIR_Validation_Real)):
    val_files_real.append(f)
print(len(val_files_real))
for f in glob('{}/*/*.wav'.format(DATA_DIR_Validation_Real_2)):
    val_files_real.append(f)
print(len(val_files_real))

L_tr_real = len(train_files_real)
# Populating the input and target data - Training
x_train_real = np.zeros((L_tr_real, MAX_FRAME, 1), dtype=np.float32)
y_train_real = np.zeros((L_tr_real, N_CLASS), dtype=np.uint8)
for i in range(L_tr_real):
    f_train = train_files_real[i]
    x_train_real[i] = get_wave(f_train)
    label = f_train[len(DATA_DIR_Train_Real) + 1:]
    label = label[:label.index('\\')]
    label_idx = LABELS.index(label[0])
    y_train_real[i, label_idx] = 1

L_val_real = len(val_files_real)
# Populating the input and target data - Validation
x_val_real = np.zeros((L_val_real, MAX_FRAME, 1), dtype=np.float32)
y_val_real = np.zeros((L_val_real, N_CLASS), dtype=np.uint8)
for i in range(L_val_real):
    f_val = val_files_real[i]
    x_val_real[i] = get_wave(f_val)
    label = f_val[len(DATA_DIR_Validation_Real) + 1:]
    label = label[:label.index('\\')]
    label_idx = LABELS.index(label[0])
    y_val_real[i, label_idx] = 1

# Unfreezing the trained network for fine-tuning

In [ ]:
set_trainable = False
for layer in reconstructed_model.layers:
    if layer.name == 'Global_Pooling':
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

# Fine-tuning model

In [ ]:
callbacks = [keras.callbacks.ModelCheckpoint("best_model.h5", save_best_only=True, monitor="val_loss"),
             keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.2, patience=3, min_lr=0.0001),
             keras.callbacks.EarlyStopping(monitor="val_loss", patience=15, verbose=1),]
history = reconstructed_model.fit(x_train_real, y_train_real, 
                    validation_data = (x_val_real, y_val_real), 
                    epochs=60, batch_size=64, verbose=1,
                    callbacks=callbacks)

# Fine-tuned model accuracy and loss

In [ ]:
training_acc   = history.history['accuracy']
validation_acc = history.history['val_accuracy']

plt.plot(training_acc[8:])
plt.plot(validation_acc[8:])
plt.title('fine-tuning model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='lower right')
plt.show()
# summarize history for loss

training_loss   = history.history['loss']
validation_loss = history.history['val_loss']

plt.plot(training_loss[15:])
plt.plot(validation_loss[15:])
plt.title('fine-tuning model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper right')
plt.show()

# Test set evaluation on fine-tuned model

In [ ]:
reconstructed_model = keras.models.load_model('best_model.h5')

L_test = len(test_files_real)
x_test = np.zeros((L_test, MAX_FRAME, 1), dtype=np.float32)
y_test = np.zeros((L_test, N_CLASS), dtype=np.uint8)
for i in range(L_test):
    f_test = test_files_real[i]
    x_test[i] = get_wave(f_test)
    label = f_test[len(DATA_DIR_Test_Real) + 1:]
    label = label[:label.index('\\')]
    label_idx = LABELS.index(label[0])
    y_test[i, label_idx] = 1


y_pred = reconstructed_model.predict(x_test) #Prediction with test data

y_pred_uni8 = np.zeros((L_test, N_CLASS), dtype=np.uint8)

for i in range(L_test):
    maxElement = np.amax(y_pred[i])
    result = np.where(y_pred[i] == np.amax(y_pred[i]))
    y_pred_uni8[i, result[0]] = 1



# Confusion Matrix of Real Test set

In [ ]:
matrix = confusion_matrix(y_test.argmax(axis=1), y_pred_uni8.argmax(axis=1))

acc_class_0 = 100*matrix[0,0]/np.sum(matrix[0,:])
acc_class_1 = 100*matrix[1,1]/np.sum(matrix[1,:])
acc_class_2 = 100*matrix[2,2]/np.sum(matrix[2,:])
acc_class_3 = 100*matrix[3,3]/np.sum(matrix[3,:])
acc_class_4 = 100*matrix[4,4]/np.sum(matrix[4,:])
print('Accuracy of class 0: %' + str(round(acc_class_0,2)))
print('Accuracy of class 1: %' + str(acc_class_1))
print('Accuracy of class 2: %' + str(acc_class_2))
print('Accuracy of class 3: %' + str(acc_class_3))
print('Accuracy of class 4: %' + str(acc_class_4))
total = (acc_class_0 + acc_class_1 +acc_class_2 +acc_class_3 +acc_class_4)/5
print('Total Test Accuracy: %' + str(total))

# Binary Classificatin Result (low bubble grade vs. high bubble grade)

In [ ]:
# Low Class
low_bubble_class_count = 0
total_low_bubble = 0
for i in range(3):
    for j in range(3):
        low_bubble_class_count = low_bubble_class_count + matrix[i,j]
for i in range(3):
    for j in range(len(matrix)):
        total_low_bubble = total_low_bubble + matrix[i,j]
print('Low bubble class accuracy: %', 100*low_bubble_class_count/total_low_bubble)
################################################################################
# High Class
high_bubble_class_count = 0
total_high_bubble = 0
for i in [3, 4]:
    for j in [3, 4]:
        high_bubble_class_count = high_bubble_class_count + matrix[i,j]
for i in [3, 4]:
    for j in range(len(matrix)):
        total_high_bubble = total_high_bubble + matrix[i,j]
print('High bubble class accuracy: %', 100*high_bubble_class_count/total_high_bubble)

print('Average Accuracy: %', 100*((high_bubble_class_count/total_high_bubble)/2+(low_bubble_class_count/total_low_bubble)/2))